# Favorita

## The Goal

The goal is to build a machine learning model that can predict the unit sales for various items sold at different Favorita stores with higher accuracy, using a training dataset that includes information about dates, stores, items, promotions, and unit sales.

## Kaggle notebook setup

Let's start to import some library...

In [1]:
import zipfile
from IPython.display import display, Markdown, HTML
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import category_encoders as ce
import os
from sklearn.model_selection import train_test_split
import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import VotingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import make_scorer

... and verify that we have data in input folder.

In [2]:
for dirname, _, filenames in os.walk("/input"):
    for filename in filenames:
        print(os.path.join(dirname, filename))

Scripts

For sake of clarify, we put here several functions that that perform different tasks related to data preparation, regression modeling, and data analysis.
<ul>
        <li><b>getMemoryUsage(df)</b>: Given a pandas DataFrame df, this function returns the total memory usage of the DataFrame in bytes. </li>
        <li><b>makeEmptyDf(df)</b>: Given a pandas DataFrame df, this function returns an empty DataFrame with the same column names as df.</li>
        <li><b>showDistinctValues(colName, dtFrame)</b>: Given a column name colName and a pandas DataFrame dtFrame, this function returns the number of distinct values in the specified column.</li>
        <li><b>showMissingData(df)</b>: Given a pandas DataFrame df, this function displays the columns that contain missing data (i.e., NaN values), and for each such column, it calls the showPercentageNan(colName, dtFrame) function to display the percentage of missing values in that column.</li>
        <li><b>showMissingDataPerCol(df, colName)</b>: Given a pandas DataFrame df and a column name colName, this function displays the number of missing values in the specified column and the percentage of missing values in that column, using the showPercentageNan(colName, dtFrame) function.</li>
        <li><b>showPercentageNan(colName, dtFrame)</b>: Given a column name colName and a pandas DataFrame dtFrame, this function displays the percentage of missing values in the specified column.</li>
        <li><b>basicEDA(df, title)</b>: Given a pandas DataFrame df and a string title, this function displays some basic information about the DataFrame, including its shape, data types, and summary statistics. If the DataFrame contains any missing values, it calls the showMissingData(df) function to display information about the missing values.</li>
        <li><b>getDistinctValues(df, lstFeatures)</b>: Given a pandas DataFrame df and a list of column names lstFeatures, this function displays the distinct values in each specified column.</li>
        <li><b>delColumn(df, colName)</b>: Given a pandas DataFrame df and a column name colName, this function removes the specified column from the DataFrame.</li>
        <li><b>reduce_mem_usage(df)</b>: Given a pandas DataFrame df, this function attempts to reduce the memory usage of the DataFrame by downcasting numeric columns to smaller data types where possible. It then displays information about the memory usage before and after the optimization.</li>
        <li><b>handleWithLinearRegression(df, colNameWithNa)</b>: This function handles missing values in a specified column of the input dataframe df using linear regression. The function first creates a copy of the input dataframe and converts any datetime columns to integers. The function then separates the rows with missing values in the specified column and the rows without missing values, and prepares the data for regression. It fits a linear regression model on the training data, predicts the missing values, and fills the missing values with the predictions. Finally, the function computes and prints the R2 and RMSE metrics for evaluating the regression performance and returns the filled dataframe and the number of missing values in the specified column. </li>
        <li><b>BinEncCategory(df, lstCols)</b>: This function applies binary encoding to categorical columns specified in lstCols of the input dataframe df. </li>
        <li><b>MSE(y_true,y_pred)</b>: This function computes and prints the mean squared error (MSE) between the true and predicted values. </li>
        <li><b>R2(y_true,y_pred)</b>: This function computes and prints the R2 score between the true and predicted values. </li>
        <li><b>two_score(y_true,y_pred)</b>: This function calls the MSE and R2 functions and returns the R2 score. </li>
        <li><b>two_scorer()</b>: This function returns a scorer object for use in cross-validation, which uses the two_score function to compute the score. </li>
        <li><b>executeVoter(X_train, X_test, y_train, y_test, whatToPredict)</b>: This function fits a voting regressor model using the specified models and returns the trained model. It also prints the estimators used in the voting regressor. </li>
        <li><b>makeCrossValidation(model, X_train, y_train, scoring)</b>: This function performs cross-validation on the input model using the specified scoring method and returns the results. </li>
        <li><b>fillTypeHolidayColumn(df)</b>: This function fills missing values in the "type_Holiday" column of the input dataframe df. It iterates over each row of the dataframe, and for each row with a missing value in the "type_Holiday" column, it checks whether the "date" column corresponds to a weekend or workday and fills the missing value accordingly. </li>
</ul>

In [3]:
def getMemoryUsage(df):
    return df.memory_usage().sum() # in bytes

def makeEmptyDf(df):
    return df[0:0] 

def showDistinctValues(colName, dtFrame):
    return dtFrame.groupby([colName]).size()

def showMissingData(df):
    colNan = list(df[df.columns[df.isna().any()]].columns)
    display(f"The columns with missing data are: {colNan}")

    for col in colNan:
        showPercentageNan(col, df)
    return

def showMissingDataPerCol(df, colName):
    nrNa = df[colName].isna().sum()
    display(f"The '{colName}' column has {nrNa} missing data")
    showPercentageNan(colName, df)

def showPercentageNan(colName, dtFrame):
    nr = dtFrame[colName].isnull().sum()
    perc = (nr / dtFrame.shape[0])
    print(f"Percent of missing '{colName}' records is {round(perc * 100,3)} % ({nr} values on {dtFrame.shape[0]} total)")
    return

def basicEDA(df, title):
    display(Markdown("**Just first five rows**"))
    display(df.head(3))
    display(f"The {title} data set consists of {df.shape[1]} different features which for {df.shape[0]} samples.")
    display(Markdown("**Info about the index dtype and columns, non-null values and memory usage.**"))
    display(df.info())
     
    # isnull() is just an alias of the isna() method as shown in pandas source code.
    nrNa = df.isna().sum()
    display(Markdown("**Count na values**"))
    display(nrNa)

    if nrNa.any() > 0:
        showMissingData(df)
        
        #colNan = list(df[df.columns[df.isna().any()]].columns)
        #display(f"The columns with missing data are: {colNan}")

        #for col in colNan:
        #    showPercentageNan(col, df)

    display(Markdown("**Show the statistic report of the numeric features of the dataset**"))
    display(df.describe(datetime_is_numeric=True).transpose())   
    return

def getDistinctValues(df, lstFeatures):
    display(Markdown(f"Distint values in:"))
    for category in lstFeatures:
        dist = sorted(df[category].unique())
        display(Markdown(f"{category} -> {dist}"))


def delColumn(df, colName):
    for name in df.columns:
        if name == colName:
            df.drop(colName, axis=1, inplace=True)
    return

# from https://www.mikulskibartosz.name/how-to-reduce-memory-usage-in-pandas/
# (adapted)
def reduce_mem_usage(df):
    start_mem = df.memory_usage().sum() / 1024**2
    print(df.info())
    print("----------------")
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype
        if col_type != object:
                c_min = df[col].min()
                c_max = df[col].max()
                if str(col_type)[:3] == 'int':
                    if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                        df[col] = df[col].astype(np.int8)
                    elif c_min > np.iinfo(np.uint8).min and c_max < np.iinfo(np.uint8).max:
                        df[col] = df[col].astype(np.uint8)
                    elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                        df[col] = df[col].astype(np.int16)
                    elif c_min > np.iinfo(np.uint16).min and c_max < np.iinfo(np.uint16).max:
                        df[col] = df[col].astype(np.uint16)
                    elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                        df[col] = df[col].astype(np.int32)
                    elif c_min > np.iinfo(np.uint32).min and c_max < np.iinfo(np.uint32).max:
                        df[col] = df[col].astype(np.uint32)                    
                    elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                        df[col] = df[col].astype(np.int64)
                    elif c_min > np.iinfo(np.uint64).min and c_max < np.iinfo(np.uint64).max:
                        df[col] = df[col].astype(np.uint64)
                elif str(col_type)[:5] == 'float':
                    if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                        df[col] = df[col].astype(np.float16)
                    elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                        df[col] = df[col].astype(np.float32)
                    else:
                        df[col] = df[col].astype(np.float64)
        elif col_type == object:
            df[col] = df[col].astype("category")

    end_mem = df.memory_usage().sum() / 1024**2
    print('\nMemory usage AFTER optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    return df

def handleWithLinearRegression(df, colNameWithNa):
    dftmp = df.copy()
    cols = list(dftmp.columns)
    for col in cols:
        if "datetime" in str(dftmp.dtypes[col]):
            dftmp[col] = dftmp[col].view(int)

    # Separating missing or nan values rows
    test = dftmp[dftmp[colNameWithNa].isna()] # Assume the Age column contains missing values.
    train = dftmp.dropna()
    
    makeEmptyDf(dftmp)
    del dftmp
    
    # Preparing df
    X_train = train.drop([colNameWithNa], axis=1)
    y_train = train[colNameWithNa]
    
    X_test = test.drop([colNameWithNa], axis=1)
    
    # Getting ready a model to predict missing values(Genre column)
    linear = LinearRegression()
    linear.fit(X_train, y_train)
    # Predictions
    y_pred = linear.predict(X_test)
    # filling the missing values with predictions
    df.loc[df[colNameWithNa].isnull(), colNameWithNa] = y_pred
    
    # Running Evaluation Metrics
    predictions = linear.predict(X_test)
    r2 = r2_score(y_pred, predictions)
    rmse = mean_squared_error(y_pred, predictions, squared=False)
   
    print('The r2 is: ', r2)
    print('The rmse is: ', rmse)
    return df, df[colNameWithNa].isna().sum()


def BinEncCategory(df, lstCols):
    encoder = ce.BinaryEncoder(cols = lstCols)
    return encoder.fit_transform(df)

# see: https://stats.stackexchange.com/questions/110599/how-to-get-both-mse-and-r2-from-a-sklearn-gridsearchcv
def MSE(y_true,y_pred):
    mse = mean_squared_error(y_true, y_pred)
    print("MSE: %2.3f" % mse)
    
    return mse

def R2(y_true,y_pred):    
    r2 = r2_score(y_true, y_pred)
    print("R2: %2.3f" % r2) 
    return r2
    
def two_score(y_true,y_pred):    
    MSE(y_true,y_pred) # set score here and not below if using MSE in GridCV
    score = R2(y_true,y_pred)
    return score

def two_scorer():
    return make_scorer(two_score, greater_is_better=True) # change for false if using MSE

def executeVoter(X_train, X_test, y_train, y_test, whatToPredict):
    models = [
        ['lr', LinearRegression()],
        ['rfr', RandomForestRegressor()],
        ['lgb', lgb.LGBMRegressor()],
        ['xgb', xgb.XGBRegressor()]
    ]

    voter_regr = VotingRegressor(models)
    voter_regr.fit(X_train, y_train)
    voter_regr.score(X_test, y_test)

    voter_regr.predict(X_test[:whatToPredict])
    print(f"This are the estimators:\n {voter_regr.named_estimators_}")
    
    return voter_regr

def makeCrossValidation(model, X_train, y_train, scoring):
    # https://www.kaggle.com/code/cesarsupo/tps-09-votingregressor
    from sklearn.model_selection import cross_validate

    return cross_validate(model, X_train, y_train, cv=2, n_jobs=-1,
                         scoring=scoring,
                         return_estimator = True
    )

def fillTypeHolidayColumn(df):
    for index, row in df.iterrows():
        if pd.isnull(df.loc[index, "type_Holiday"]):
            test = pd.to_datetime(df.loc[index,"date"]).strftime("%A")
            if test == "Sunday" or test == "Saturday":
                df.at[index,"type_Holiday"] = "Weekend"
            else:
                df.at[index,"type_Holiday"] = "Work Day"
    return df

Let's enable automatic garbage collection to free automatically memory that is no longer in use by the program.

In [4]:
import gc
gc.enable()

#  Import data sets <a name="importdataset"></a>

We have available a variety of data sources, each of which provides valuable insights into different aspects of the problem at hand.

The Holidays and Events dataset contains metadata that can be used to understand past sales trends and seasonal patterns, although it does require some additional manipulation to make it useful. 

Similarly, the Daily Oil Price data is particularly relevant to our analysis, given that Ecuador's economy is heavily influenced by oil prices, and we can use it to identify which product families are most impacted by fluctuations in oil prices. 

Meanwhile, the Stores data provides information on the location and type of each store, which can help us better understand regional differences in sales patterns. 

Finally, the Transaction data is particularly useful, as it is highly correlated with the sales column in the training data, and can be used to identify patterns and trends in store sales over time. 

Speaking of the training data, it consists of time series data for each store and product family combination, and includes a sales column that provides the total sales for each product family at a particular store on a given date. 
And a test dataset will be used to evaluate the performance of our models.

So define paths...

In [5]:
pathHolidays = "./input/holidays_events.csv"
pathOilPrices = "./input/oil.csv"
pathStores = "./input/stores.csv"
pathTransactions = "./input/transactions.csv"
pathTrain = "./input/train.csv"
pathTest = "./input/test.csv"

Clean up old csv datasets...

In [6]:
for dirname, _, filenames in os.walk("./input"):
    for filename in filenames:
        if filename.endswith(('csv')):
            os.remove(os.path.join(dirname, filename))

...and extract the new ones.

In [7]:
with zipfile.ZipFile("./input/input.zip","r") as z:
    z.extractall("./input")

... and load them in separate Panda's data frame objects.

In [8]:
dfHolidays = pd.read_csv(pathHolidays, sep=",", parse_dates=['date'])
dfHolidays.Name = "Holidays"

dfOilPrices = pd.read_csv(pathOilPrices, sep=",", parse_dates=['date'])
dfOilPrices.Name = "OilPrices"

dfStores = pd.read_csv(pathStores, sep=",")
dfStores.Name = "Stores"

dfTransactions = pd.read_csv(pathTransactions, sep=",", parse_dates=['date'])
dfTransactions.Name = "transaction"

dfTrain = pd.read_csv(pathTrain, sep=",", parse_dates=['date'])
dfTrain.Name = "Train"

dfTest = pd.read_csv(pathTest, sep=",", parse_dates=['date'])
dfTest.Name = "Test"

# Profiling datasets <a name="profilingdatasets"></a>

### Profiling Train and Test <a name="profilingtraintest"></a>

In [9]:
basicEDA(dfTrain, "Train")

**Just first five rows**

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0


'The Train data set consists of 6 different features which for 3000888 samples.'

**Info about the index dtype and columns, non-null values and memory usage.**

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 6 columns):
 #   Column       Dtype         
---  ------       -----         
 0   id           int64         
 1   date         datetime64[ns]
 2   store_nbr    int64         
 3   family       object        
 4   sales        float64       
 5   onpromotion  int64         
dtypes: datetime64[ns](1), float64(1), int64(3), object(1)
memory usage: 137.4+ MB


None

**Count na values**

id             0
date           0
store_nbr      0
family         0
sales          0
onpromotion    0
dtype: int64

**Show the statistic report of the numeric features of the dataset**

,count,mean,min,25%,50%,75%,max,std
id,3.00089e+06,1.50044e+06,0,750222,1.50044e+06,2.25067e+06,3.00089e+06,866282
date,3000888,2015-04-24 08:27:04.703216128,2013-01-01 00:00:00,2014-02-26 18:00:00,2015-04-24 12:00:00,2016-06-19 06:00:00,2017-08-15 00:00:00,NaN
store_nbr,3.00089e+06,27.5,1,14,27.5,41,54,15.5858
sales,3.00089e+06,357.776,0,0,11,195.847,124717,1102
onpromotion,3.00089e+06,2.60277,0,0,0,0,741,12.2189


We can see there is no missing data. And the distinct values of family are:

In [10]:
getDistinctValues(dfTrain, ["family"])

Distint values in:

family -> ['AUTOMOTIVE', 'BABY CARE', 'BEAUTY', 'BEVERAGES', 'BOOKS', 'BREAD/BAKERY', 'CELEBRATION', 'CLEANING', 'DAIRY', 'DELI', 'EGGS', 'FROZEN FOODS', 'GROCERY I', 'GROCERY II', 'HARDWARE', 'HOME AND KITCHEN I', 'HOME AND KITCHEN II', 'HOME APPLIANCES', 'HOME CARE', 'LADIESWEAR', 'LAWN AND GARDEN', 'LINGERIE', 'LIQUOR,WINE,BEER', 'MAGAZINES', 'MEATS', 'PERSONAL CARE', 'PET SUPPLIES', 'PLAYERS AND ELECTRONICS', 'POULTRY', 'PREPARED FOODS', 'PRODUCE', 'SCHOOL AND OFFICE SUPPLIES', 'SEAFOOD']

Same thing about test data.

In [11]:
basicEDA(dfTest, "Test")

**Just first five rows**

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2


'The Test data set consists of 5 different features which for 28512 samples.'

**Info about the index dtype and columns, non-null values and memory usage.**

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28512 entries, 0 to 28511
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   id           28512 non-null  int64         
 1   date         28512 non-null  datetime64[ns]
 2   store_nbr    28512 non-null  int64         
 3   family       28512 non-null  object        
 4   onpromotion  28512 non-null  int64         
dtypes: datetime64[ns](1), int64(3), object(1)
memory usage: 1.1+ MB


None

**Count na values**

id             0
date           0
store_nbr      0
family         0
onpromotion    0
dtype: int64

**Show the statistic report of the numeric features of the dataset**

,count,mean,min,25%,50%,75%,max,std
id,28512,3.01514e+06,3.00089e+06,3.00802e+06,3.01514e+06,3.02227e+06,3.0294e+06,8230.85
date,28512,2017-08-23 12:00:00.000363776,2017-08-16 00:00:00,2017-08-19 18:00:00,2017-08-23 12:00:00,2017-08-27 06:00:00,2017-08-31 00:00:00,NaN
store_nbr,28512,27.5,1,14,27.5,41,54,15.5861
onpromotion,28512,6.96538,0,0,0,6,646,20.684


holidays
	datetime64, object, bool
	no na values
oilprices
	datetime64,	float64
	some prices missed (3.53%)
stores
	int64, object
	no na values
transactions
	datetime64, int64
	no na values
train /test
	int64, datetime64[ns], object, float64, int64
	no missed data

### Profiling Holidays <a name="profilingholidays"></a>

In [12]:
basicEDA(dfHolidays, "Holidays")

**Just first five rows**

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False


'The Holidays data set consists of 6 different features which for 350 samples.'

**Info about the index dtype and columns, non-null values and memory usage.**

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date         350 non-null    datetime64[ns]
 1   type         350 non-null    object        
 2   locale       350 non-null    object        
 3   locale_name  350 non-null    object        
 4   description  350 non-null    object        
 5   transferred  350 non-null    bool          
dtypes: bool(1), datetime64[ns](1), object(4)
memory usage: 14.1+ KB


None

**Count na values**

date           0
type           0
locale         0
locale_name    0
description    0
transferred    0
dtype: int64

**Show the statistic report of the numeric features of the dataset**

,count,mean,min,25%,50%,75%,max
date,350,2015-04-24 00:45:15.428571392,2012-03-02,2013-12-23 06:00:00,2015-06-08,2016-07-03,2017-12-26


We can see there is no missing data. And the distinct values of locale, locale_name and type are:

In [13]:
getDistinctValues(dfHolidays, ["locale", "locale_name", "type"])

Distint values in:

locale -> ['Local', 'National', 'Regional']

locale_name -> ['Ambato', 'Cayambe', 'Cotopaxi', 'Cuenca', 'Ecuador', 'El Carmen', 'Esmeraldas', 'Guaranda', 'Guayaquil', 'Ibarra', 'Imbabura', 'Latacunga', 'Libertad', 'Loja', 'Machala', 'Manta', 'Puyo', 'Quevedo', 'Quito', 'Riobamba', 'Salinas', 'Santa Elena', 'Santo Domingo', 'Santo Domingo de los Tsachilas']

type -> ['Additional', 'Bridge', 'Event', 'Holiday', 'Transfer', 'Work Day']

### Profiling Oil prices <a name="profilingoilprices"></a>

In [14]:
basicEDA(dfOilPrices, "Oil prices")

**Just first five rows**

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97


'The Oil prices data set consists of 2 different features which for 1218 samples.'

**Info about the index dtype and columns, non-null values and memory usage.**

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1218 entries, 0 to 1217
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        1218 non-null   datetime64[ns]
 1   dcoilwtico  1175 non-null   float64       
dtypes: datetime64[ns](1), float64(1)
memory usage: 19.2 KB


...

date           0
dcoilwtico    43
dtype: int64

"The columns with missing data are: ['dcoilwtico']"

Percent of missing 'dcoilwtico' records is 3.53 % (43 values on 1218 total)


**Show the statistic report of the numeric features of the dataset**

,count,mean,min,25%,50%,75%,max,std
date,1218,2015-05-02 12:00:00.000012544,2013-01-01 00:00:00,2014-03-03 06:00:00,2015-05-02 12:00:00,2016-06-30 18:00:00,2017-08-31 00:00:00,NaN
dcoilwtico,1175,67.7144,26.19,46.405,53.19,95.66,110.62,25.6305


We've found 43 dcoilwtico missing values, so let's handling missing data using Linear Regression.

In [15]:
dfOilPrices, verifyScore = handleWithLinearRegression(dfOilPrices, "dcoilwtico")
print(verifyScore) # overfitting?

The r2 is:  1.0
The rmse is:  0.0
0


### Profiling Stores <a name="profilingstores"></a>

In [16]:
basicEDA(dfStores, "Stores")

**Just first five rows**

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8


'The Stores data set consists of 5 different features which for 54 samples.'

**Info about the index dtype and columns, non-null values and memory usage.**

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   store_nbr  54 non-null     int64 
 1   city       54 non-null     object
 2   state      54 non-null     object
 3   type       54 non-null     object
 4   cluster    54 non-null     int64 
dtypes: int64(2), object(3)
memory usage: 2.2+ KB


None

**Count na values**

store_nbr    0
city         0
state        0
type         0
cluster      0
dtype: int64

**Show the statistic report of the numeric features of the dataset**

,count,mean,std,min,25%,50%,75%,max
store_nbr,54.0,27.500000,15.732133,1.0,14.25,27.5,40.75,54.0
cluster,54.0,8.481481,4.693395,1.0,4.00,8.5,13.00,17.0


We have no missing data here!

In [17]:
getDistinctValues(dfStores, ["city", "state", "type"])

Distint values in:

city -> ['Ambato', 'Babahoyo', 'Cayambe', 'Cuenca', 'Daule', 'El Carmen', 'Esmeraldas', 'Guaranda', 'Guayaquil', 'Ibarra', 'Latacunga', 'Libertad', 'Loja', 'Machala', 'Manta', 'Playas', 'Puyo', 'Quevedo', 'Quito', 'Riobamba', 'Salinas', 'Santo Domingo']

state -> ['Azuay', 'Bolivar', 'Chimborazo', 'Cotopaxi', 'El Oro', 'Esmeraldas', 'Guayas', 'Imbabura', 'Loja', 'Los Rios', 'Manabi', 'Pastaza', 'Pichincha', 'Santa Elena', 'Santo Domingo de los Tsachilas', 'Tungurahua']

type -> ['A', 'B', 'C', 'D', 'E']

### Profiling Transactions <a name="profilingtransaction"></a>

In [18]:
basicEDA(dfTransactions, "Trasactions")

**Just first five rows**

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358


'The Trasactions data set consists of 3 different features which for 83488 samples.'

**Info about the index dtype and columns, non-null values and memory usage.**

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 83488 entries, 0 to 83487
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          83488 non-null  datetime64[ns]
 1   store_nbr     83488 non-null  int64         
 2   transactions  83488 non-null  int64         
dtypes: datetime64[ns](1), int64(2)
memory usage: 1.9 MB


None

**Count na values**

date            0
store_nbr       0
transactions    0
dtype: int64

**Show the statistic report of the numeric features of the dataset**

,count,mean,min,25%,50%,75%,max,std
date,83488,2015-05-20 16:07:40.866227968,2013-01-01 00:00:00,2014-03-27 00:00:00,2015-06-08 00:00:00,2016-07-14 06:00:00,2017-08-15 00:00:00,NaN
store_nbr,83488,26.9392,1,13,27,40,54,15.6082
transactions,83488,1694.6,5,1046,1393,2079,8359,963.287


We have no missing data here!

# Handling train dataset <a name="handlingtraindataset"></a>

Let's merge combines multiple datasets into a single DataFrame called dfTrainMerged and dfTestMerged!

In [19]:
dfTrainMerged = dfTrain.merge(dfStores, on = 'store_nbr', how='left') \
                     .merge(dfTransactions, left_on=["date", "store_nbr"], right_on=["date", "store_nbr"], how="left") \
                     .merge(dfHolidays, left_on="date", right_on="date", how="left") \
                     .rename(columns={"type_x": "type_Store"}) \
                     .rename(columns={"type_y": "type_Holiday"}) \
                     .drop_duplicates(subset="id").copy() \
                     .merge(dfOilPrices, left_on="date", right_on="date", how="left").copy()

In [20]:
dfTestMerged = dfTest.merge(dfStores, on = 'store_nbr', how='left') \
                     .merge(dfTransactions, left_on=["date", "store_nbr"], right_on=["date", "store_nbr"], how="left") \
                     .merge(dfHolidays, left_on="date", right_on="date", how="left") \
                     .rename(columns={"type_x": "type_Store"}) \
                     .rename(columns={"type_y": "type_Holiday"}) \
                     .drop_duplicates(subset="id").copy() \
                     .merge(dfOilPrices, left_on="date", right_on="date", how="left").copy()

Clean up memory!

In [21]:
del dfHolidays 
del dfOilPrices 
del dfStores 
del dfTransactions
del dfTrain
del dfTest
gc.collect()

0

## Feature engineering <a name="11111"></a>

We conduct a missing values analysis to see what is the percentage of missing per column.

In [22]:
showMissingData(dfTrainMerged)

"The columns with missing data are: ['transactions', 'type_Holiday', 'locale', 'locale_name', 'description', 'transferred', 'dcoilwtico']"

Percent of missing 'transactions' records is 8.19 % (245784 values on 3000888 total)
Percent of missing 'type_Holiday' records is 85.036 % (2551824 values on 3000888 total)
Percent of missing 'locale' records is 85.036 % (2551824 values on 3000888 total)
Percent of missing 'locale_name' records is 85.036 % (2551824 values on 3000888 total)
Percent of missing 'description' records is 85.036 % (2551824 values on 3000888 total)
Percent of missing 'transferred' records is 85.036 % (2551824 values on 3000888 total)
Percent of missing 'dcoilwtico' records is 28.563 % (857142 values on 3000888 total)


In [23]:
showMissingData(dfTestMerged)

"The columns with missing data are: ['transactions', 'type_Holiday', 'locale', 'locale_name', 'description', 'transferred', 'dcoilwtico']"

Percent of missing 'transactions' records is 100.0 % (28512 values on 28512 total)
Percent of missing 'type_Holiday' records is 93.75 % (26730 values on 28512 total)
Percent of missing 'locale' records is 93.75 % (26730 values on 28512 total)
Percent of missing 'locale_name' records is 93.75 % (26730 values on 28512 total)
Percent of missing 'description' records is 93.75 % (26730 values on 28512 total)
Percent of missing 'transferred' records is 93.75 % (26730 values on 28512 total)
Percent of missing 'dcoilwtico' records is 25.0 % (7128 values on 28512 total)


First of all, we can get rid some feature, as showed below:

In [24]:
delColumn(dfTrainMerged, "locale")
delColumn(dfTrainMerged, "locale_name")
delColumn(dfTrainMerged, "description")
delColumn(dfTrainMerged, "transferred")
gc.collect()
dfTrainMerged.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type_Store,cluster,transactions,type_Holiday,dcoilwtico
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13,NaN,Holiday,104.896636
1,1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13,NaN,Holiday,104.896636
2,2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13,NaN,Holiday,104.896636
3,3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13,NaN,Holiday,104.896636
4,4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13,NaN,Holiday,104.896636


In [25]:
delColumn(dfTestMerged, "locale")
delColumn(dfTestMerged, "locale_name")
delColumn(dfTestMerged, "description")
delColumn(dfTestMerged, "transferred")
gc.collect()
dfTestMerged.head()

,id,date,store_nbr,family,onpromotion,city,state,type_Store,cluster,transactions,type_Holiday,dcoilwtico
0,3000888,2017-08-16,1,AUTOMOTIVE,0,Quito,Pichincha,D,13,NaN,NaN,46.8
1,3000889,2017-08-16,1,BABY CARE,0,Quito,Pichincha,D,13,NaN,NaN,46.8
2,3000890,2017-08-16,1,BEAUTY,2,Quito,Pichincha,D,13,NaN,NaN,46.8
3,3000891,2017-08-16,1,BEVERAGES,20,Quito,Pichincha,D,13,NaN,NaN,46.8
4,3000892,2017-08-16,1,BOOKS,0,Quito,Pichincha,D,13,NaN,NaN,46.8


We use fillTypeHolidayColumn() function which iterates over each row in the DataFrame, and for any rows where the "type_Holiday" value is missing (i.e. NaN), it determines whether the corresponding date is a weekday or weekend. If the date is a weekend (Saturday or Sunday), it fills in the "type_Holiday" column with the value "Weekend". Otherwise, it fills in the "type_Holiday" column with the value "Work Day". That on train and test merged data frames.

In [26]:
dfTrainMerged = fillTypeHolidayColumn(dfTrainMerged)
dfTestMerged = fillTypeHolidayColumn(dfTestMerged)

A quick check!

In [27]:
showMissingDataPerCol(dfTrainMerged, "type_Holiday")

"The 'type_Holiday' column has 0 missing data"

Percent of missing 'type_Holiday' records is 0.0 % (0 values on 3000888 total)


In [28]:
showMissingDataPerCol(dfTestMerged, "type_Holiday")

"The 'type_Holiday' column has 0 missing data"

Percent of missing 'type_Holiday' records is 0.0 % (0 values on 28512 total)


Let's handle categorical data by using "Binary Encoding" that is a technique used to encode categorical features as binary digits (0s and 1s). Each distinct value in the categorical feature is represented by a unique binary code, which is then split into multiple binary columns.

In [29]:
dfTrainMerged = BinEncCategory(dfTrainMerged, ["type_Holiday","family", "city", "state", "type_Store"])
dfTrainMerged.head()

,id,date,store_nbr,family_0,family_1,family_2,family_3,family_4,family_5,sales,...,state_4,type_Store_0,type_Store_1,type_Store_2,cluster,transactions,type_Holiday_0,type_Holiday_1,type_Holiday_2,dcoilwtico
0,0,2013-01-01,1,0,0,0,0,0,1,0.0,...,1,0,0,1,13,NaN,0,0,1,104.896636
1,1,2013-01-01,1,0,0,0,0,1,0,0.0,...,1,0,0,1,13,NaN,0,0,1,104.896636
2,2,2013-01-01,1,0,0,0,0,1,1,0.0,...,1,0,0,1,13,NaN,0,0,1,104.896636
3,3,2013-01-01,1,0,0,0,1,0,0,0.0,...,1,0,0,1,13,NaN,0,0,1,104.896636
4,4,2013-01-01,1,0,0,0,1,0,1,0.0,...,1,0,0,1,13,NaN,0,0,1,104.896636


In [30]:
dfTestMerged = BinEncCategory(dfTestMerged, ["type_Holiday","family", "city", "state", "type_Store"])
dfTestMerged.head()

,id,date,store_nbr,family_0,family_1,family_2,family_3,family_4,family_5,onpromotion,...,state_3,state_4,type_Store_0,type_Store_1,type_Store_2,cluster,transactions,type_Holiday_0,type_Holiday_1,dcoilwtico
0,3000888,2017-08-16,1,0,0,0,0,0,1,0,...,0,1,0,0,1,13,NaN,0,1,46.8
1,3000889,2017-08-16,1,0,0,0,0,1,0,0,...,0,1,0,0,1,13,NaN,0,1,46.8
2,3000890,2017-08-16,1,0,0,0,0,1,1,2,...,0,1,0,0,1,13,NaN,0,1,46.8
3,3000891,2017-08-16,1,0,0,0,1,0,0,20,...,0,1,0,0,1,13,NaN,0,1,46.8
4,3000892,2017-08-16,1,0,0,0,1,0,1,0,...,0,1,0,0,1,13,NaN,0,1,46.8


Now we can delete labeled features...

In [31]:
delColumn(dfTrainMerged, "type_Holiday")
delColumn(dfTrainMerged, "family")
delColumn(dfTrainMerged, "city")
delColumn(dfTrainMerged, "state")
delColumn(dfTrainMerged, "type_Store")

delColumn(dfTestMerged, "type_Holiday")
delColumn(dfTestMerged, "family")
delColumn(dfTestMerged, "city")
delColumn(dfTestMerged, "state")
delColumn(dfTestMerged, "type_Store")
gc.collect()

0

It is better change datas in float32 type.

In [32]:
dfTrainMerged[("date")] = dfTrainMerged[("date")].values.astype("float32")
dfTestMerged[("date")] = dfTestMerged[("date")].values.astype("float32")

Now we want to fill NaN transactions values. To do that we can simply get the mean value and use it.

In [33]:
transactions_mean = dfTrainMerged["transactions"].mean()
transactions_mean

1694.6021583940208

In [34]:
dfTrainMerged["transactions"].fillna(transactions_mean,inplace=True)
dfTrainMerged.head()

,id,date,store_nbr,family_0,family_1,family_2,family_3,family_4,family_5,sales,...,state_4,type_Store_0,type_Store_1,type_Store_2,cluster,transactions,type_Holiday_0,type_Holiday_1,type_Holiday_2,dcoilwtico
0,0,1.356998e+18,1,0,0,0,0,0,1,0.0,...,1,0,0,1,13,1694.602158,0,0,1,104.896636
1,1,1.356998e+18,1,0,0,0,0,1,0,0.0,...,1,0,0,1,13,1694.602158,0,0,1,104.896636
2,2,1.356998e+18,1,0,0,0,0,1,1,0.0,...,1,0,0,1,13,1694.602158,0,0,1,104.896636
3,3,1.356998e+18,1,0,0,0,1,0,0,0.0,...,1,0,0,1,13,1694.602158,0,0,1,104.896636
4,4,1.356998e+18,1,0,0,0,1,0,1,0.0,...,1,0,0,1,13,1694.602158,0,0,1,104.896636


Unfortunately, we have no data in transactions feature.

In [35]:
test_transactions_mean = dfTestMerged["transactions"].mean()
test_transactions_mean

nan

So we use the train transaction mean value...

In [36]:
dfTestMerged["transactions"].fillna(transactions_mean,inplace=True)
dfTestMerged.head()

,id,date,store_nbr,family_0,family_1,family_2,family_3,family_4,family_5,onpromotion,...,state_3,state_4,type_Store_0,type_Store_1,type_Store_2,cluster,transactions,type_Holiday_0,type_Holiday_1,dcoilwtico
0,3000888,1.502842e+18,1,0,0,0,0,0,1,0,...,0,1,0,0,1,13,1694.602158,0,1,46.8
1,3000889,1.502842e+18,1,0,0,0,0,1,0,0,...,0,1,0,0,1,13,1694.602158,0,1,46.8
2,3000890,1.502842e+18,1,0,0,0,0,1,1,2,...,0,1,0,0,1,13,1694.602158,0,1,46.8
3,3000891,1.502842e+18,1,0,0,0,1,0,0,20,...,0,1,0,0,1,13,1694.602158,0,1,46.8
4,3000892,1.502842e+18,1,0,0,0,1,0,1,0,...,0,1,0,0,1,13,1694.602158,0,1,46.8


Let's fill "dcoilwtico" missing values. We choose two ways: by Linear Regression and, on test data simply put into the mean values on test data (remember that "Percent of missing 'dcoilwtico' records is 25.0 % (7128 values on 28512 total)").

In [37]:
dfTrainMerged, countMissing = handleWithLinearRegression(dfTrainMerged, "dcoilwtico")
print(countMissing)

The r2 is:  1.0
The rmse is:  0.0
0


In [38]:
dfTestMerged["dcoilwtico"].fillna(dfTestMerged["dcoilwtico"].mean(),inplace=True)
dfTestMerged.head()

,id,date,store_nbr,family_0,family_1,family_2,family_3,family_4,family_5,onpromotion,...,state_3,state_4,type_Store_0,type_Store_1,type_Store_2,cluster,transactions,type_Holiday_0,type_Holiday_1,dcoilwtico
0,3000888,1.502842e+18,1,0,0,0,0,0,1,0,...,0,1,0,0,1,13,1694.602158,0,1,46.8
1,3000889,1.502842e+18,1,0,0,0,0,1,0,0,...,0,1,0,0,1,13,1694.602158,0,1,46.8
2,3000890,1.502842e+18,1,0,0,0,0,1,1,2,...,0,1,0,0,1,13,1694.602158,0,1,46.8
3,3000891,1.502842e+18,1,0,0,0,1,0,0,20,...,0,1,0,0,1,13,1694.602158,0,1,46.8
4,3000892,1.502842e+18,1,0,0,0,1,0,1,0,...,0,1,0,0,1,13,1694.602158,0,1,46.8


In [39]:
showMissingData(dfTrainMerged)

'The columns with missing data are: []'

In [40]:
showMissingData(dfTestMerged)

'The columns with missing data are: []'

At this point, we can reduce data size. In fact we have:

In [41]:
print(getMemoryUsage(dfTrainMerged))

732216672


So reduce it!

In [42]:
dfTrainMerged = reduce_mem_usage(dfTrainMerged)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3000888 entries, 0 to 3000887
Data columns (total 30 columns):
 #   Column          Dtype  
---  ------          -----  
 0   id              int64  
 1   date            float32
 2   store_nbr       int64  
 3   family_0        int64  
 4   family_1        int64  
 5   family_2        int64  
 6   family_3        int64  
 7   family_4        int64  
 8   family_5        int64  
 9   sales           float64
 10  onpromotion     int64  
 11  city_0          int64  
 12  city_1          int64  
 13  city_2          int64  
 14  city_3          int64  
 15  city_4          int64  
 16  state_0         int64  
 17  state_1         int64  
 18  state_2         int64  
 19  state_3         int64  
 20  state_4         int64  
 21  type_Store_0    int64  
 22  type_Store_1    int64  
 23  type_Store_2    int64  
 24  cluster         int64  
 25  transactions    float64
 26  type_Holiday_0  int64  
 27  type_Holiday_1  int64  
 28  type_Holiday

In [43]:
print(getMemoryUsage(dfTestMerged))

6500736


In [44]:
dfTestMerged = reduce_mem_usage(dfTestMerged)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28512 entries, 0 to 28511
Data columns (total 28 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              28512 non-null  int64  
 1   date            28512 non-null  float32
 2   store_nbr       28512 non-null  int64  
 3   family_0        28512 non-null  int64  
 4   family_1        28512 non-null  int64  
 5   family_2        28512 non-null  int64  
 6   family_3        28512 non-null  int64  
 7   family_4        28512 non-null  int64  
 8   family_5        28512 non-null  int64  
 9   onpromotion     28512 non-null  int64  
 10  city_0          28512 non-null  int64  
 11  city_1          28512 non-null  int64  
 12  city_2          28512 non-null  int64  
 13  city_3          28512 non-null  int64  
 14  city_4          28512 non-null  int64  
 15  state_0         28512 non-null  int64  
 16  state_1         28512 non-null  int64  
 17  state_2         28512 non-null 

In [45]:
print(getMemoryUsage(dfTestMerged))

1283040


# Train and test the model

In [46]:
X = dfTrainMerged.drop("sales", axis=1)
y = dfTrainMerged["sales"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)

In [47]:
makeEmptyDf(dfTrainMerged) # may be redundant!
del dfTrainMerged
gc.collect()

0

We've selected four regression models (Linear Regression, Random Forest Regressor, LGBM Regressor, and XGB Regressor), In order to choose what to use, we call executeVoter() function which creates an instance of the VotingRegressor class, that is a type of ensemble model that combines the predictions of multiple base estimators using a weighted average. The VotingRegressor object fits the X_train and y_train data and then scores the model with the X_test and y_test data. After that, it makes predictions on X_test data for the number of samples specified by whatToPredict. Finally, it prints the names of the models used for the ensemble.

In [48]:
voter_regr = executeVoter(X_train, X_test, y_train, y_test, 5)
for name, models in voter_regr.named_estimators_.items():
    print(f"{name} -> {models.score(X_test, y_test)}")

This are the estimators:
 {'lr': LinearRegression(), 'rfr': RandomForestRegressor(), 'lgb': LGBMRegressor(), 'xgb': XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=6,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)}
lr -> 0.22351857622593385
rfr -> 0.9531138061875539
lgb -> 0.9073586757994937
xgb -> 0.8801303120269346


In [49]:
del voter_regr
gc.collect()

132

We choose XGBRegressor.

In [50]:
xgbr = xgb.XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1,
             monotone_constraints='()', n_estimators=100, n_jobs=6,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=1)

In order to verify if our model work properly. we call cross_val_score() function that returns a dictionary with the results of the cross-validation, including the score for each fold, the time taken for each fold, and the model instance trained on each fold.

In [51]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(xgbr, X_train, y_train, cv=10)
print("Mean cross-validation score: %.2f" % scores.mean())

Mean cross-validation score: 0.88


So, we can fit.

In [52]:
xgbr.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
             gamma=0, gpu_id=-1, importance_type=None,
             interaction_constraints='', learning_rate=0.300000012,
             max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=100, n_jobs=6,
             num_parallel_tree=1, predictor='auto', random_state=0, reg_alpha=0,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=1)

Check scores.

In [53]:
scoreTraining = xgbr.score(X_train, y_train)
scoreTest = xgbr.score(X_test, y_test)

In [54]:
print("Training score: ", scoreTraining)
print("Test score: ", scoreTest)

Training score:  0.8836883801863323
Test score:  0.8801303120269346


It's time to make predictions!

In [55]:
# Predict target for test data
predictions = xgbr.predict(X_test)
predictions = predictions.reshape(len(predictions), 1)

from sklearn.metrics import mean_absolute_error

# Error printing
mse = mean_squared_error(y_test, predictions)
rmse = mse**.5

print(f"Mean Absolute Error:\t {mean_absolute_error(y_test, predictions)}")
print(f"rmse:\t {rmse}")
print(f"Mean Square Error  :\t {rmse}")

Mean Absolute Error:	 155.99464416503906
rmse:	 382.53933621263053
Mean Square Error  :	 382.53933621263053


### Submit predictions

In [56]:
dfTestMerged['sales'] = np.NaN
predictions = xgbr.predict(dfTestMerged)
predictions

array([2718.6587, 2639.9302, 1380.7925, ..., 2884.8293, 2657.1685,
       2744.6748], dtype=float32)

Activate the following code only in Kagle environment, because if output folder isn't empty, we have a "Submission CSV Not Found" error!

In [57]:
# for dirname, _, filenames in os.walk("/kaggle/working/"):
#     for filename in filenames:
#         os.remove(os.path.join(dirname, filename))

In [58]:
myId = dfTestMerged['id']

submission = pd.DataFrame({"id": myId,"sales": predictions})
submission.to_csv('submission.csv', index=False)
